In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

<h3>Récupération des urls de toutes les pages</h3>

In [2]:
def getAllPages():

    urls = []
    actualPage = 1

    for i in range(163) :
        i = f"https://www.barreaudenice.com/annuaire/avocats/?fwp_paged={actualPage}"
        actualPage += 1
        urls.append(i)
        
    return urls

<h3>Récupération de toutes les infos concernants les avocats sur 1 seule page</h3>

In [3]:
def getInfo(url) :

    lawyer = requests.get(url)
    soup = bs(lawyer.text,"lxml")
    list_lawyer = []
    
    name = soup.find_all('div', class_="callout secondary annuaire-single")
    for i in name:
    
        try:
            if i.find('span', class_="telephone") :
                phone_number_all = i.find('span', class_="telephone").get_text().strip()
            else :
                phone_number_all = 'NaN'
        except:
                phone_number_all = 'NaN'
    
        try:
            if i.find('span', class_="fax") :
                fax_all = i.find('span', class_="fax").get_text().strip()
            else :
                fax_all = 'NaN'
        except:
                fax_all = 'NaN'
            
        try:
            if i.find('span', class_="email") :
                email_all = i.find('span', class_="email").get_text().strip()
            else :
                email_all = 'NaN'
        except:
                email_all = 'NaN'

        try:
            if i.find('span', class_="num-case") :
                num_case_all = i.find('span', class_="num-case").get_text().strip()
            else :
                num_case_all = 'NaN'
        except:
                num_case_all = 'NaN'

        try:
            if i.find('span', class_="adresse") :
                location_all = i.find('span', class_="adresse").get_text().strip()
            else :
                location_all = 'NaN'
        except:
                location_all = 'NaN'

        try:
            if i.find('h3', class_="nom-prenom") :
                name_all = i.find('h3', class_="nom-prenom").get_text().strip()
            else :
                name_all = 'NaN'
        except:
                name_all = 'NaN'

        try:
            if i.find('span', class_="date") :
                date_all = i.find('span', class_="date").get_text().strip()
            else :
                date_all = 'NaN'
        except:
                date_all = 'NaN'
  
        info = [name_all, date_all, location_all, num_case_all, email_all, phone_number_all, fax_all]
        list_lawyer.append(info)

    return list_lawyer
       

<h3>Répeter l'étape précendente sur toutes les pages du site</h3>

In [4]:
def getAllLawyer():
    
    page = getAllPages()
    allInfo = []
    
    for i in page:
            list = getInfo(i)
            allInfo.extend(list)

    return allInfo


<h3>Transformer toutes la data en file CSV</h3>

In [ ]:
data = getAllLawyer()

dt = pd.DataFrame(data, columns = ['name', 'date', 'location', 'num-case', 'email', 'phone-number', 'fax'])
#dt.to_csv("lawyerData(dirty).csv")

dt

<h3>Nettoyage</h3>

In [ ]:
backup = dt

In [ ]:
dt.isnull().sum()

In [ ]:
#dt.dt['fax'] == 'NaN'

In [ ]:
dt

In [ ]:
dt['date'] = pd.to_datetime(dt['date'])

In [ ]:
dt.dtypes

In [ ]:
dt.groupby(['fax']).describe()

In [ ]:
dt['fax'] = dt['fax'].str.replace('F . ','')
dt['fax'] = dt['fax'].str.replace("+33) ", "")
dt['fax'] = dt['fax'].str.replace("+33 ", "0")
dt['fax'] = dt['fax'].str.replace(".", " ")
dt['fax'] = dt['fax'].str.replace("(", "")
dt['fax'] = dt['fax'].str.replace("109", "09")
dt['fax'] = dt['fax'].str.replace("NaN", "0")

In [ ]:
dt.groupby("phone-number").describe()

In [ ]:
dt['phone-number'] = dt['phone-number'].str.replace('T . ','')
dt['phone-number'] = dt['phone-number'].str.replace('.',' ')
dt['phone-number'] = dt['phone-number'].str.replace('T . ','')
dt['phone-number'] = dt['phone-number'].str.replace('+33(0)', '0')
dt['phone-number'] = dt['phone-number'].str.replace('(+33)', '')
dt['phone-number'] = dt['phone-number'].str.replace('+33', '0')

In [ ]:
dt.drop(dt[ dt['phone-number'].str.contains('--', na=False) ].index, inplace=True)

In [ ]:
dt

In [ ]:
dt['num-case'] = dt['num-case'].str.replace('N° de Case : ', "")
dt['num-case'] = dt['num-case'].str.replace('NaN', "0")

In [ ]:
dt.groupby("num-case").describe()

In [ ]:
dt

In [ ]:
dt['num-case'] = pd.to_numeric(dt['num-case'], errors='coerce').astype("Int64")

In [ ]:
dt.dtypes

In [ ]:
dt.groupby('email').describe()

In [ ]:
dt['email'] = dt["email"].str.replace('\n ', '')
dt['email'] = dt['email'].str.replace("Email : ", "")

In [ ]:
dt

In [ ]:
dt.groupby("location").describe()

In [ ]:
dt["location"] = dt["location"].str.replace('\r\n', " ")
dt["location"] = dt["location"].str.replace('\r', " ")
dt["location"] = dt["location"].str.replace('\n', " ")

In [ ]:
dt.insert(3, "zip-code", 0)

In [ ]:
dt.insert(4,'city', "Nice")

In [ ]:
dt['zip-code'] = dt['location'].str.extract(r'\b(06\d{3})\b') # if he was something starting with 06 ...
dt

In [ ]:
dt['zip-code'] = pd.to_numeric(dt['zip-code'], errors='coerce').astype("Int64")

In [ ]:
dt

In [ ]:
dt.dtypes